In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
### PRE_PROCESSING ###

def load_images_and_create_label(folder, new_pixels):
    
    """
    Loads images from 
    Resize image to 'new_pixels'
    Returns array of images and array of labels
    """
    
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.resize(img, (new_pixels, new_pixels))
            images.append(img)
            if filename.find("cat") == -1:
                # dog 
                labels.append(1)
            else:
                # cat
                labels.append(0)
    return images, labels

In [3]:
def layer_sizes(X, Y):
    
    """
    Defines sizes of input, hidden and output layers
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    
    n_X = X.shape[0]
    n_Y = Y.shape[0]
    n_H = 4
    
    return (n_X, n_H, n_Y)

In [4]:
def init_parameters(n_X, n_H, n_Y):
    
    """
    Initializes parameters with required shapes
    Returns:
    param - Python dictionary containing parameters:
    W1, b1, W2, b2
    """
    
    W1 = np.random.randn(n_H, n_X) * 0.01
    b1 = np.zeros((n_H, 1))
    W2 = np.random.randn(n_Y, n_H) * 0.01
    b2 = np.zeros((n_Y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [5]:
def sigmoid(z):
    
    """
    Sigmoid non-linearity
    """
    return 1 / (1 + np.exp(-z))

In [6]:
def forward_propagation(X, parameters):
    
    """
    Performs forward propogation.
    Returns:
    cache - Python dictionary conataining values:
    Z1, A1, Z2, A2
    """
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [7]:
def compute_cost(A2, Y):
    
    """
    Computes cross-entropy cost.
    """
    
    # number of examples
    m = Y.shape[1]

    logprobs = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))
    cost = - np.sum(logprobs) / m
    
    # makes sure cost is the dimension we expect. E.g., turns [[17]] into 17 
    cost = np.squeeze(cost)

    return cost

In [8]:
def backward_propagation(X, Y, parameters, cache):
    
    """
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    
    # number of examples
    m = X.shape[1]
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
    A1 = cache["A1"]
    A2 = cache["A2"]
    
    dZ2 = A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [9]:
def update_parameters(parameters, grads, learning_rate):
    
    """
    Updates parameters using the gradient descent update 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [10]:
def nn_model(X, Y, n_H, num_iterations = 10000, learning_rate = 1.2, print_cost = False):
    
    """
    Define the NN model.
    Returns:
    Updated parameters to be used for prediction
    """
    
    np.random.seed(3)
    n_X = layer_sizes(X, Y)[0]
    n_Y = layer_sizes(X, Y)[2]
    
    parameters = init_parameters(n_X, n_H, n_Y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(X, parameters)
        
        cost = compute_cost(A2, Y)
 
        grads = backward_propagation(X, Y, parameters, cache)
 
        parameters = update_parameters(parameters, grads, learning_rate)
     
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))

    return parameters

In [28]:
def predict(parameters, X, labels):
    
    """
    Predicts based on learned parameters.
    """
    
    A2, cache = forward_propagation(X, parameters)
    predictions = np.round(A2)
    
    print("Accuracy: {} %".format(100 - np.mean(np.abs(predictions - labels)) * 100))

In [12]:
# load train images and labels
images_train_dataset,labels_train_dataset = load_images_and_create_label('/Users/rahulmadhyan/Documents/AI/Neural Networks/Vanilla NN/Data/train', 64)
images_train_dataset = np.array(images_train_dataset)
labels_train_dataset = np.array(labels_train_dataset)

# load test images and labels
images_test_dataset,labels_test_dataset = load_images_and_create_label('/Users/rahulmadhyan/Documents/AI/Neural Networks/Vanilla NN/Data/test1', 64)
images_test_dataset = np.array(images_test_dataset)
labels_test_dataset = np.array(labels_test_dataset)

In [33]:
# reduce size of dataset
size_train = 5000
images_train = images_train_dataset[:size_train]
labels_train = labels_train_dataset[:size_train]

size_test = 5000
images_test = images_test_dataset[:size_test]
labels_test = labels_test_dataset[:size_test]

# flatten dataset
images_train = images_train.reshape(images_train.shape[0], -1).T
labels_train = labels_train.reshape(labels_train.shape[0], -1).T
images_test = images_test.reshape(images_test.shape[0], -1).T
labels_test = labels_test.reshape(labels_test.shape[0], -1).T

#standaradize the dataset
images_train = images_train / 255
images_test = images_test / 255

In [ ]:
updated_parameters = nn_model(images_train, labels_train, 8, num_iterations = 15000, learning_rate = 0.005, print_cost = True)

Cost after iteration 0: 0.692935
Cost after iteration 1000: 0.687523
Cost after iteration 2000: 0.672080
Cost after iteration 3000: 0.653530
Cost after iteration 4000: 0.630028
Cost after iteration 5000: 0.600388
Cost after iteration 6000: 0.593852
Cost after iteration 7000: 0.574343
Cost after iteration 8000: 0.554644
Cost after iteration 9000: 0.535157
Cost after iteration 10000: 0.516339
Cost after iteration 11000: 0.498318
Cost after iteration 12000: 0.480699
Cost after iteration 13000: 0.463338


In [34]:
# training accuracy
predict(updated_parameters, images_train, labels_train)

Accuracy: 72.08 %


In [35]:
# test accuracy
predict(updated_parameters, images_test, labels_test)

Accuracy: 71.36 %
